<a href="https://colab.research.google.com/gist/almagashi/0cbfa5f5d205a579a5c4bbb6161d7c56/grocery_store___final_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import heapq
import random
import scipy.stats as sts
import numpy as np
import matplotlib.pyplot as plt


class Event:
    '''
    Store the properties of one event in the Schedule class defined below. Each
    event has a time at which it needs to run, a function to call when running
    the event, along with the arguments and keyword arguments to pass to that
    function.
    '''

    def __init__(self, timestamp, function, *args, **kwargs):
        self.timestamp = timestamp
        self.function = function
        self.args = args
        self.kwargs = kwargs

    def __lt__(self, other):
        '''
        This overloads the less-than operator in Python. We need it so the
        priority queue knows how to compare two events. We want events with
        earlier (smaller) times to go first.
        '''
        return self.timestamp < other.timestamp

    def run(self, schedule):
        '''
        Run an event by calling the function with its arguments and keyword
        arguments. The first argument to any event function is always the
        schedule in which events are being tracked. The schedule object can be
        used to add new events to the priority queue.
        '''
        self.function(schedule, *self.args, **self.kwargs)


class Schedule:
    '''
    Implement an event schedule using a priority queue. You can add events and
    run the next event.

    The `now` attribute contains the time at which the last event was run.
    '''

    def __init__(self):
        self.now = 0  # Keep track of the current simulation time
        self.priority_queue = []  # The priority queue of events to run

    def add_event_at(self, timestamp, function, *args, **kwargs):
        # Add an event to the schedule at a particular point in time.
        heapq.heappush(
            self.priority_queue,
            Event(timestamp, function, *args, **kwargs))

    def add_event_after(self, interval, function, *args, **kwargs):
        # Add an event to the schedule after a specified time interval.
        self.add_event_at(self.now + interval, function, *args, **kwargs)

    def next_event_time(self):
        return self.priority_queue[0].timestamp

    def run_next_event(self):
        # Get the next event from the priority queue and run it.
        event = heapq.heappop(self.priority_queue)
        self.now = event.timestamp
        event.run(self)

    def __repr__(self):
        return (
            f'Schedule() at time {self.now} ' +
            f'with {len(self.priority_queue)} events in the queue')

    def print_events(self):
        print(repr(self))
        for event in sorted(self.priority_queue):
            print(f'   {event.timestamp}: {event.function.__name__}')

class Manager:
  '''
  Implements manager queue. This is just an example of another queue. 
  '''
  def __init__(self, service_distribution):
      self.service_distribution = service_distribution
      # We start with an empty queue and the server not busy
      self.people_in_queue = 0
      self.people_being_served = 0

  def add_customer(self, schedule):
      # Add the customer to the queue
      self.people_in_queue += 1
      if self.people_being_served < 1:
          # This customer can be served immediately
          schedule.add_event_after(0, self.start_serving_customer)

  def start_serving_customer(self, schedule):
      # Move the customer from the queue to a server
      self.people_in_queue -= 1
      self.people_being_served += 1
      # Schedule when the server will be done with the customer
      schedule.add_event_after(
          self.service_distribution.rvs(),
          self.finish_serving_customer)

  def finish_serving_customer(self, schedule):
      # Remove the customer from the server
      self.people_being_served -= 1
      if self.people_in_queue > 0:
          # There are more people in the queue so serve the next customer
          schedule.add_event_after(0, self.start_serving_customer)


class Queue:
  '''
  Creates all the cashier' queues. Since the number of the cashiers and queues 
  are the same, then the queues created is equal to cashier_count. It also 
  contains the manager attribute in the initialization.
  '''
  def __init__(self, service_distribution, manager, chance_to_see_manager=0.05):
      # the cashier service distribution
      self.service_distribution = service_distribution
      # initialize the manager
      self.manager = manager
      # since only 5% of the people end up seeing the manager, then 
      # we can define it as each individual has a 5% chance of seeing the manager
      self.chance_to_see_manager=chance_to_see_manager 
      # We start with an empty queue and the server not busy
      self.people_in_queue = 0
      self.people_being_served = 0

  def add_customer(self, schedule):
      # Add the customer to the queue
      self.people_in_queue += 1
      if self.people_being_served < 1:
          # This customer can be served immediately
          schedule.add_event_after(0, self.start_serving_customer)


  def start_serving_customer(self, schedule):
      # Move the customer from the queue to a server
      self.people_in_queue -= 1
      self.people_being_served += 1
      # Schedule when the server will be done with the customer
      schedule.add_event_after(
          self.service_distribution.rvs(),
          self.finish_serving_customer)

  def finish_serving_customer(self, schedule):
      # Remove the customer from the server
      self.people_being_served -= 1
      # Should we send them to the manager with a 5% chance
      if random.random() < self.chance_to_see_manager:
          ## Send to the manager
          schedule.add_event_after(0, self.manager.start_serving_customer)
      if self.people_in_queue > 0:
          # There are more people in the queue so serve the next customer
          schedule.add_event_after(0, self.start_serving_customer)

  def __lt__(self, other):
      '''
        This overloads the less-than operator in Python. We need it so the
        priority queue knows how to compare two queues. We want the queue with
        the shortest length to be chosen first.
      '''
      return self.people_in_queue < other.people_in_queue

  def __radd__(self, other):
      '''
        This overloads the less-than operator in Python. We need it so the
        priority queue knows how to compare two queues. We want the queue with
        the shortest length to be chosen first.
      '''
      return self.people_in_queue + other


class GroceryStore2:
  '''
  Assigns the distributions of arrival and service for both cashiers and managers,
  while taking the cashier count as 1. It also allows customers to join the 
  shortest queue.
 '''

  def __init__(self, arrival_distribution, service_distribution,
                 manager_distribution, cashier_count=1):
    # initialize the Manager and cashiers
      self.manager = Manager(manager_distribution)
      self.cashiers = []
    # for each cashier, add a queue with a specific arrival distribution
      for _ in range(cashier_count):
          self.cashiers.append(Queue(service_distribution, self.manager))
      self.arrival_distribution = arrival_distribution

  def add_customer(self, schedule):
      # Add this customer to the shortest queue
      heapq.heapify(self.cashiers)
      shortest_queue = self.cashiers [0]
      shortest_queue.add_customer(schedule)

      # Schedule when to add another customer
      schedule.add_event_after(
          self.arrival_distribution.rvs(),
          self.add_customer)

  def run(self, schedule):
      # Schedule when the first customer arrives
      schedule.add_event_after(
          self.arrival_distribution.rvs(),
          self.add_customer)




In [8]:
def run_simulation(arrival_distribution, service_distribution, 
                   manager_service_distribution, cashier_count, run_until):
    schedule = Schedule()
    grocery_store = GroceryStore2(arrival_distribution, service_distribution, 
                                  manager_service_distribution, cashier_count)
    grocery_store.run(schedule)
    while schedule.next_event_time() < run_until:
        schedule.run_next_event()
    return grocery_store



def run_experiment(arrival_rate_list, service_time_dist,
                   manager_service_distribution, cashier_count, 
                   run_until, num_trials):
    '''
    Run an experiment with different arrival rates for an M/G/1 queue. By
    setting the service time distribution appropriately, you can also simulate
    M/D/1 and M/M/1 queues.
    '''
    
    # We record only the mean and standard error of the mean for each experiment
    results_mean = []
    results_std_err = []

    for arrival_rate in arrival_rate_list:
        arrival_distribution = sts.expon(scale=1/arrival_rate)
        maxqueuelen = 10 # the maximum queue length
        exceededmax = [] # a list to store times the maxqueuelen is exceeded
        queue_lengths = [] # normal queue lengths
        for trial in range(num_trials):
            grocery_store = run_simulation(
                arrival_distribution, service_time_dist, manager_service_distribution, cashier_count, run_until)
            queue_lengths.append(sum(grocery_store.cashiers))
            # see how many times is the maximum queue length exceeded
            # if sum(grocery_store.cashiers) > (maxqueuelen-1)*cashier_count:
            #   exceededmax.append(1)

        results_mean.append(np.mean(queue_lengths))
        results_std_err.append(sts.sem(queue_lengths))
        
    # Convert lists to arrays so we can easily add, subtract, and multiply them
    results_mean = np.array(results_mean)
    results_std_err = np.array(results_std_err)
    
    return results_mean, results_std_err

def make_error_plot(queue_type, rho, mean, std_err, theoretical_function):
    '''
    Plot the empirical mean and 95% confidence interval of the mean of the queue
    length. Also plot the theoretical value for the average queue length using the
    supplied theoretical function (a function of rho).
    '''
    
    plt.figure(figsize=(8, 6))
    plt.title(f'Average queue length for an {queue_type} queue')
    plt.xlabel('utilization ρ')
    plt.ylabel('average queue length')

    plt.errorbar(
        rho, mean, 1.96 * std_err,
        color='black', marker='o', capsize=5, linestyle='--', linewidth=1,
        label='empirical')

    plt.plot(
        rho, theoretical_function(rho),
        color='red', marker='o', linestyle='--', linewidth=1,
        label='theoretical')

    plt.legend()
    plt.show()


In [9]:
arrival_rate_list =np.array([1,1,1,1,1,1,1])
arrival_distribution = sts.expon(scale=1/1.2)
service_time_dist = sts.norm(loc=1/1, scale=0)
manager_service_distribution = sts.norm(loc=1/1, scale=0)
service_rate = 3
cashier_count = 2
run_until = 540
num_trials = 100
run_experiment(arrival_rate_list, service_time_dist, manager_service_distribution, cashier_count, run_until, num_trials)

(array([0.2 , 0.2 , 0.21, 0.3 , 0.15, 0.17, 0.14]),
 array([0.06030227, 0.0531816 , 0.05182527, 0.08587047, 0.04793725,
        0.04725816, 0.04499158]))

In [13]:
tau = 1/3
sigma = 1
np.random.seed(123)
mg1_mean, mg1_std_err = run_experiment(arrival_rate_list, service_time_dist, manager_service_distribution, cashier_count, run_until, num_trials)
print('M/G/1 experiment complete')
def theoretical_mg1(rho):
    return rho**2 / 2 / (1-rho) * (1 + sigma**2 / tau**2)
rho = 1 * tau

M/G/1 experiment complete
